In [1]:
#mon_env_gdn
import pandas as pd
import os
import re
import nltk
import numpy as np
import time
import matplotlib as plt 
import seaborn as sns
import sklearn
from tqdm.notebook import tqdm as tqdm
from pprint import pprint
import gensim
import spacy
import tweepy as tw
os.chdir(r"C:\Users\csamassa\Desktop\Mémoire\Nouveau GDN")
#os.chdir(r"C:\Users\Cheik\Desktop\mémoire")
#nltk.download('punkt')

In [3]:
start_time = time.time()

demo=pd.read_csv("DEMOCRATIE_ET_CITOYENNETE.csv", 
                 sep=",",
                 #nrows=10,
                 usecols=[0,10,2,11,13,14,16,17,19,20,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47]
                 ,dtype={"authorZipCode":object}
                 )
fisc=pd.read_csv("LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv", 
                 sep=",",
                 #nrows=10,
                 usecols=[0,10,2,11,12,13,14,15,16,17,18]
                 ,dtype={"authorZipCode":object}
                 )
eco=pd.read_csv("LA_TRANSITION_ECOLOGIQUE.csv", 
                sep=",",
                #nrows=10,               
                usecols=[0,10,2,11,12,14,16,17,18,20,22,23,24,25,26]
                ,dtype={"authorZipCode":object}
                )
                
org=pd.read_csv("ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv", 
                sep=",",
               #nrows=10,
                usecols=[0,10,2,11,13,15,16,19,20,21,24,25,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]
                ,dtype={"authorZipCode":object}
               )
print("--- %s seconds ---" % (time.time() - start_time))

--- 20.08816432952881 seconds ---


In [4]:
start_time = time.time()
# On rajoute une colonne comportant le thème de chaque question
demo.insert(column="Thème",value="DEMOCRATIE ET CITOYENNETE",loc=3)
fisc.insert(column="Thème",value="LA FISCALITE ET LES DEPENSES PUBLIQUES",loc=3)
eco.insert(column="Thème",value="LA TRANSITION ECOLOGIQUE",loc=3)
org.insert(column="Thème",value="ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES",loc=3)


# On nettoie le début des questions
def clean_question(df):
    colonnes=df.columns
    colonnes1=[re.sub(pattern=r"\bQ[A-Za-z0-9]+\s+\-\s",repl='',string=nom) for nom in colonnes]
    return(colonnes1)



# On applique tt en même temps
demo.columns,fisc.columns,eco.columns,org.columns=clean_question(demo),clean_question(fisc),clean_question(eco),clean_question(org)
col1=["id", "authorZipCode","Thème"]

def empiller(df):
    stack_0=df.loc[:, ~df.columns.isin(col1)].stack(dropna=False) # prend les QO (toutes les questions sauf celles de col1)
    stack_1=stack_0.reset_index()                                 # On supp l'index pour avoir le level 0 pour la future jointure
    stack_2=stack_1.merge(df[col1],left_on="level_0",right_index=True,how="left")
    stack_2.columns=["idx_0","Question","Réponse","id","authorZipCode","Thème"] #idx_0 c'est le numéro de la ligne dans le fichier original de chaque thème 
    stack_2.dropna(inplace=True)
    return stack_2


demo_1,fisc_1,eco_1,org_1=empiller(demo),empiller(fisc),empiller(eco),empiller(org)

del(demo,fisc,eco,org)# On supp les variables inutiles de l'environnement

contributions=pd.concat([demo_1,fisc_1,eco_1,org_1],axis=0)   #On met tout dans un même df
 
contributions.reset_index(drop=True,inplace=True)

del(demo_1,fisc_1,eco_1,org_1,col1)

contributions.drop(columns="id", inplace=True) # On supp la colonne (on a qu'à utiliser idx_0 si on veut la trace des contributions)


print("--- %s seconds ---" % (time.time() - start_time))

--- 12.559457778930664 seconds ---


In [5]:
contributions

,idx_0,Question,Réponse,authorZipCode,Thème
0,0,title,Les augmentations de rémunérations,79190,DEMOCRATIE ET CITOYENNETE
1,0,En qui faites-vous le plus confiance pour vous...,Le citoyen,79190,DEMOCRATIE ET CITOYENNETE
2,0,Y a-t-il d'autres points sur la démocratie et ...,Afin d’éviter de creuser les inégalités ne plu...,79190,DEMOCRATIE ET CITOYENNETE
3,1,title,rénover l'enquête publique pour en faire un vr...,01800,DEMOCRATIE ET CITOYENNETE
4,1,En qui faites-vous le plus confiance pour vous...,Un instrument de démocratie locale à modernise...,01800,DEMOCRATIE ET CITOYENNETE
...,...,...,...,...,...
5963582,111952,Quand vous pensez à l'évolution des services p...,"Je n'en vois pas beaucoup, mais je constate pa...",50260,ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES
5963583,111952,Quels sont les services publics qui doivent le...,"La SNCF , la renationalisation et en refaire u...",50260,ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES
5963584,111952,"Si oui, comment ?",En les laissant faire preuve de bon sens avec ...,50260,ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES
5963585,111952,"Si oui, comment ?",En simplifiant les normes tout en maintenant u...,50260,ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES


In [6]:
#start_time = time.time()

#contributions=pd.read_csv('contributions.csv',dtype={"authorZipCode":object})

#print("--- %s seconds ---" % (time.time() - start_time))

In [6]:
start_time = time.time()

def compte_group(df,nom,fichier=None,export=False):
    resultat=df.loc[df["Réponse"].str.contains(nom,case=False, regex=True)]
    ###On va mettre chaque thème dans une feuille différente
    ##D'abord on filtre les résultats par thèmes
    resultat["nb_occurr"]=resultat.loc[:,"Réponse"].str.count(pat=nom, flags=re.I)
    if export==True:
        resultat_1=resultat.loc[resultat["Thème"]=="DEMOCRATIE ET CITOYENNETE"]
        resultat_2=resultat.loc[resultat["Thème"]=="LA FISCALITE ET LES DEPENSES PUBLIQUES"]
        resultat_3=resultat.loc[resultat["Thème"]=="LA TRANSITION ECOLOGIQUE"]
        resultat_4=resultat.loc[resultat["Thème"]=="ORGANISATION DE L'ETAT ET DES SERVICES PUBLIQUES"] # je sais que c'est public mais changer ça est trop chiant à faire pzrce que faudra changer le nom des thèmes (ou y'a la faute aussi)
    
        writer = pd.ExcelWriter(fichier+'.xlsx', engine='xlsxwriter')
    
        resultat_1.to_excel(writer, sheet_name="DEMOCRATIE")
        resultat_2.to_excel(writer, sheet_name="FISCALITE")
        resultat_3.to_excel(writer, sheet_name="TRANSITION_ECOLOGIQUE")
        resultat_4.to_excel(writer, sheet_name="ORGANISATION_DE_LETAT") 
    
        writer.save()
    return(resultat)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0 seconds ---


On va prendre un exemple pour essayer notre code: toutes les contributions du GDN qui contiennent les mots pandémie, épidémie sras, coronavirus

In [7]:
start_time = time.time()
corona=compte_group(contributions,r"\bpand[e-é]mi[a-z]\b|\b[e-é]pid[e-é]mie\b|\bsras\b|\bcoronavirus\b")


print("--- %s seconds ---" % (time.time() - start_time))

#DF series.apply est plus rapide que df.apply

#On ne  prend que les contributions de plus d'une phrase


--- 71.59719014167786 seconds ---


C:\Users\csamassa\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [8]:
start_time = time.time()

aleo=corona["Réponse"].apply(nltk.sent_tokenize)
essai=pd.DataFrame(aleo.apply(pd.Series).stack()).reset_index()#la fonction series fait que le séparateur devient la virgule entre chaque éléments de la liste, le stack fait qu'on les empile, on a un multi index où chaque index de level 0= numéro de la contribution, level 1= numéro de la phrase dans la contribution

#On ajoute le nombre de phrases de chaque contributions
essai_gpby=essai.groupby(by="level_0",as_index=False)[0].count()

essai=essai.merge(essai_gpby,on="level_0")


essai.columns=["level_0","nb_sent","sent","nb_sent_total"]
contain=essai.loc[essai["sent"].str.contains(r"\bpand[e-é]mi[a-z]\b|\b[e-é]pid[e-é]mie\b|\bsras\b|\bcoronavirus\b",regex=True,case=False)]

# On met dans un dictionnaire l'index de chaque phrase contenant notre pattern
dico_lvl_contain=dict(zip(contain.index,zip(contain["nb_sent"],contain["nb_sent_total"])))

print("--- %s seconds ---" % (time.time() - start_time))

#DF series.apply est plus rapide que df.apply

#On ne  prend que les contributions de plus d'une phrase

--- 0.44719815254211426 seconds ---


Pour chaque phrase qui contient les mots qu'on veut, on prend celle d'avant et celle d'après. 
Si jamais la phrase qui contient les patterns est la première, python va renvoyer une erreur lorsqu'on va lui demander 
de prendre la phrase d'avant vu qu'il n'y a rien avant, viceversa si c'est après. 
Pour résoudre cela, on va dire à python d'exclure la première lorsqu'on lui demandera de prendre les phrases x-1 et la dernière phrase quand on lui demande de prendre les phrases x+1 (encore une fois si y'a rien après x on va avoir une erreur index out of bounds). 
On a selectionné dans une autre opérationles phrases qui contiennent le pattern, cette opération a pour but de prendre x-1 et x+1 pas x. 
Si la première phrase contient le pattern, python ne va prendre que celle qui la suit, et si dernière phrase contient le pattern, python ne va prendre que celle qui la précède, 
Si la première et la deuxième phrase contiennent le patern, python va prendre celle qui la suit (x+1), et la ligne suivante python va de nouveau prendre la première (x-1, qui contient le pattern),  et la troisème (x+1), la deuxième étant x sur cette ligne (x+1 la ligne précédente)

In [9]:
start_time = time.time()

idx_avant=[[k-1,k,v] for (k,v) in dico_lvl_contain.items() if v[0]>0] #prend la phrase d'avant si la phrase  (celle qui matche le pattern) n'est pas la première de la contribution, donc d'index supérieur à 0
phrase_avant=essai.loc[[x[0] for x in idx_avant],]
phrase_avant["id_x"]=essai.loc[[x[1] for x in idx_avant],].index # On ajoute l'index des phrases qui contiennent la phrase du patern(celle d'après du coup si répond aux conditions du for)
phrase_avant.columns=["level_0","nb_sent_av", "sent_av","nb_sent_total","id_x"]

idx_apres=[[k+1,k,v] for (k,v) in dico_lvl_contain.items() if v[0]<(v[1]-1)] # # prend la phrase d'après si la phrase n'est pas la dernière de la contribution, donc d'index inférieure au nombre de phrases dans la contribution. on ajoute moins 1 parce que v[1] c'est la taille par ex 20 et v[0] c'est l'index (0:19) donc faut mettre v[1]-1 pour que ça corresponde

phrase_apres=essai.loc[[x[0] for x in idx_apres],]
phrase_apres["id_x"]=essai.loc[[x[1] for x in idx_apres],].index #On ajoute l'index des phrases du pattern (celle d'avant du coup si matche les conditions)
phrase_apres.columns=["level_0","nb_sent_ap", "sent_ap","nb_sent_total","id_x"]

contain=contain.rename_axis('id_x').reset_index() #On ajoute la colonne de l'id pour la jointure
essai_final=contain.merge(phrase_avant, on="id_x", how="outer").merge(phrase_apres,on="id_x",how="outer") #On fait une union des trois dataframes
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.05900001525878906 seconds ---


In [33]:
essai_final.loc[:,["sent_av","sent","sent_ap"]]

,sent_av,sent,sent_ap
0,Que tous les profs les informent de l’existenc...,"Lutte contre EPIDEMIE sida, diminution nb d’av...",Tous les enfants ont les mêmes droits.
1,qui est une injustice sociale exorbitante au r...,Prévue pour faire face aux risques d'épidémie ...,Elle donne donc lieu à une immigration sanitai...
2,,Les gens ne partiraient pas si leur vie n'étai...,
3,"Ensuite la langue, les lois, les DEVOIRS.",Si leur dossier est recevable parce que la gue...,Si comme dirait le Premier Ministre Néozélanda...
4,,Il faut pourvoir distinguer les asiles d'urgen...,"Pour les autres, il faut faire appliquer des c..."
...,...,...,...
199,Le rétablissement du jour de carence suffirait...,"le taux d’absentéisme pour raisons de santé, q...",le problème tient au statut très avantageux de...
200,"ça ne se passe pas si aml que ça, sauf que de ...","aussi des congés, une épidémie et des services...",
201,Les cabinets de médecins sont débordés et la p...,Lorsqu’on a besoin d’une consultation simple (...,Il s’agirait d’un niveau « 1 » qui desorgorger...
202,,Les pénuries de médicaments de plus en plus fr...,Il faut assurer notre indépendance dans ce dom...


In [10]:
essai_final['sent_av']=essai_final['sent_av'].astype(str).replace('nan','')
essai_final['sent_ap']=essai_final['sent_ap'].astype(str).replace('nan','')


In [11]:
essai_final["phrase_av_ap"]=essai_final[['sent_av', 'sent','sent_ap']].apply(lambda x: ''.join(x), axis=1)


In [27]:
# Stopwords et punct removal perso, mais on va plutôt utiliser celui de Spacy
'''
def stop_words(liste=False):
    stopwords=pd.read_csv("stopwords.csv",sep=';')
    nvx_stpwds=pd.DataFrame(liste,dtype=str)
    stopwords=pd.concat([stopwords["a"],nvx_stpwds[0]],axis=0,ignore_index=True)
    stopwords=pd.concat([stopwords,stopwords.str.upper(),stopwords.str.capitalize()],axis=0, ignore_index=True)
    
    return(stopwords.tolist())
stopwords=stop_words(['oui','non','faut','faudrait','une','qu','il'])


essai_final["text_processed"] = essai_final["phrase_av_ap"].map(lambda x: re.sub('[,\.!?:]', ' ', x))
essai_final["text_processed"]=essai_final["text_processed"].map(lambda x:' '.join([word for word in x.split() if word not in (stopwords)]))#On enlève les stopwords

'''

In [3]:
from spacy import displacy
import fr_core_news_sm
from spacy.lemmatizer import Lemmatizer
from spacy.lang.fr.stop_words import STOP_WORDS
nlp = fr_core_news_sm.load()
#stop_list=["oui","non","faut","faudrait","une","qu","il"]
nlp = fr_core_news_sm.load()

#nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.

for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True
    
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)


In [5]:
doc = nlp(u"Paris est une ville très chère.")
print(doc)

['Paris', 'ville']


In [49]:
dir(nlp)

['Defaults',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_format_docs_and_golds',
 '_meta',
 '_multiprocessing_pipe',
 '_optimizer',
 '_path',
 'add_pipe',
 'begin_training',
 'create_pipe',
 'disable_pipes',
 'entity',
 'evaluate',
 'factories',
 'from_bytes',
 'from_disk',
 'get_pipe',
 'has_pipe',
 'lang',
 'linker',
 'make_doc',
 'matcher',
 'max_length',
 'meta',
 'parser',
 'path',
 'pipe',
 'pipe_factories',
 'pipe_labels',
 'pipe_names',
 'pipeline',
 'preprocess_gold',
 'rehearse',
 'remove_pipe',
 'rename_pipe',
 'replace_pipe',
 'resume_training',
 'tagger',
 'tensorizer',
 'to_bytes',
 'to_disk',
 'tokenizer',
 'update',
 'use_params',
 'v

In [23]:
doc_list = []
entité=[]
# Iterates through each article in the corpus.
for doc in tqdm(essai_final["phrase_av_ap"]):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)    
    doc_list.append(pr)

In [25]:
for ent in pr.ents:
        entité.append(ent.text, ent.start_char, ent.label_)

['injustice',
 'social',
 'exorbitante',
 'regard',
 'situation',
 'français',
 'matière',
 'santé',
 'prévue',
 'faire',
 'face',
 'risque',
 'épidémie',
 'véhiculer',
 'migrant',
 'attribuer',
 'pathologie',
 'aucunement',
 'contagieux',
 'donne',
 'lieu',
 'immigration',
 'sanitaire',
 'onéreux',
 'communauté']

In [17]:
ents=[]
for doc in tqdm(essai_final["phrase_av_ap"]):
    
    for ent in doc.ents:
        prc=nlp(ent)
        prc.append(ent.text, ent.start_char, ent.label_)

AttributeError: 'list' object has no attribute 'ents'

In [14]:
doc_list[9]

['culte',
 'pré-carré',
 'n',
 'héler',
 'réserver',
 'élite',
 '.c',
 'pandémie',
 'grave',
 'vaccin',
 'sérum',
 'réseau',
 'sociau',
 'anonymat',
 'détruire',
 'société',
 'exprimer',
 'crainte',
 'représaille',
 'juridique',
 'bon',
 'nombre',
 'faux',
 'idée',
 'nuisible',
 'bêtise',
 'illimiter']

In [24]:
dictionnaire = gensim.corpora.Dictionary(doc_list) # dictionnaire des tokens uniques

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [25]:
corpus = [dictionnaire.doc2bow(doc) for doc in doc_list]

In [26]:
doc_130 = corpus[130]
for i in range(len(doc_130)):
    print("Le mot {} (\"{}\") aparait {} fois.".format(doc_130[i][0], 
                                               dictionnaire[doc_130[i][0]], 
doc_130[i][1]))

Le mot 55 ("faire") aparait 1 fois.
Le mot 72 ("épidémie") aparait 1 fois.
Le mot 110 ("exemple") aparait 1 fois.
Le mot 120 ("vrai") aparait 1 fois.
Le mot 567 ("attendre") aparait 1 fois.
Le mot 577 ("remplacer") aparait 1 fois.
Le mot 1042 ("prendre") aparait 1 fois.
Le mot 1125 ("hiver") aparait 1 fois.
Le mot 1162 ("maladie") aparait 1 fois.
Le mot 1309 ("totalement") aparait 1 fois.
Le mot 1591 ("plante") aparait 1 fois.
Le mot 1659 ("local") aparait 2 fois.
Le mot 1711 ("insecte") aparait 1 fois.
Le mot 1722 ("biodiversité") aparait 1 fois.
Le mot 1728 ("dengue") aparait 1 fois.
Le mot 1870 ("flore") aparait 1 fois.
Le mot 1906 ("Australie") aparait 1 fois.
Le mot 1907 ("bloquer") aparait 1 fois.
Le mot 1908 ("chikungunya?et") aparait 1 fois.
Le mot 1909 ("chose?une") aparait 1 fois.
Le mot 1910 ("exotique") aparait 1 fois.
Le mot 1911 ("grosse") aparait 1 fois.
Le mot 1912 ("importation") aparait 1 fois.
Le mot 1913 ("moustique") aparait 2 fois.
Le mot 1914 ("préserve") aparait

In [12]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [39]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionnaire,
                                           num_topics=3, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [40]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.019*"épidémie" + 0.012*"l" + 0.009*"faire" + 0.009*"d" + 0.007*"pouvoir" '
  '+ 0.007*"n" + 0.007*"cas" + 0.007*"devoir" + 0.005*"  " + 0.005*"maladie"'),
 (1,
  '0.011*"  " + 0.011*"épidémie" + 0.007*"gestion" + 0.007*"administratif" + '
  '0.007*"n" + 0.007*"devoir" + 0.007*"faire" + 0.006*"l" + 0.005*"falloir" + '
  '0.005*"pandémie"'),
 (2,
  '0.017*"épidémie" + 0.012*"l" + 0.010*"  " + 0.010*"d" + 0.007*"faire" + '
  '0.005*"n" + 0.005*"santé" + 0.005*"y" + 0.004*"falloir" + '
  '0.004*"politique"')]


In [32]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    pprint(doc)
    break


[(0, 0.05924481494731159),
 (1, 0.1321443107530509),
 (2, 0.1418129870198082),
 (3, 0.1418129870198082),
 (4, 0.1418129870198082),
 (5, 0.1321443107530509),
 (6, 0.1418129870198082),
 (7, 0.07805288262146544),
 (8, 0.1418129870198082),
 (9, 0.1321443107530509),
 (10, 0.1418129870198082),
 (11, 0.31128290243047163),
 (12, 0.1418129870198082),
 (13, 0.1321443107530509),
 (14, 0.1418129870198082),
 (15, 0.1048898431085945),
 (16, 0.1418129870198082),
 (17, 0.1418129870198082),
 (18, 0.12464470279943649),
 (19, 0.1418129870198082),
 (20, 0.2856635005255116),
 (21, 0.1418129870198082),
 (22, 0.1418129870198082),
 (23, 0.1321443107530509),
 (24, 0.1418129870198082),
 (25, 0.10134879271045097),
 (26, 0.1418129870198082),
 (27, 0.1321443107530509),
 (28, 0.2836259740396164),
 (29, 0.1418129870198082),
 (30, 0.1418129870198082),
 (31, 0.1418129870198082),
 (32, 0.1418129870198082),
 (33, 0.10011945646355355),
 (34, 0.1418129870198082),
 (35, 0.05079841497700909),
 (36, 0.1418129870198082),
 (37

In [33]:
###LDA_multicore
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionnaire, passes=2, workers=2)


In [34]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.018*"épidémie" + 0.012*"l" + 0.009*"pouvoir" + 0.009*"d" + 0.008*"cancer" + 0.008*"devoir" + 0.008*"faire" + 0.007*"	 " + 0.007*"grippe" + 0.007*"mettre"
Topic: 1 
Words: 0.018*"épidémie" + 0.009*"devoir" + 0.009*"  " + 0.006*"limiter" + 0.006*"chasse" + 0.006*"vie" + 0.005*"l" + 0.005*"aide" + 0.005*"n" + 0.005*"faire"
Topic: 2 
Words: 0.021*"épidémie" + 0.010*"l" + 0.009*"pouvoir" + 0.008*"faire" + 0.008*"falloir" + 0.008*"n" + 0.007*"pandémie" + 0.006*"vie" + 0.006*"y" + 0.005*"aller"
Topic: 3 
Words: 0.021*"  " + 0.016*"épidémie" + 0.012*"gestion" + 0.011*"administratif" + 0.009*"l" + 0.009*"santé" + 0.008*"faire" + 0.008*"social" + 0.006*"devoir" + 0.005*"n"
Topic: 4 
Words: 0.032*"d" + 0.019*"l" + 0.016*"épidémie" + 0.009*"n" + 0.007*"devoir" + 0.007*"qu" + 0.006*"faire" + 0.006*"enfant" + 0.006*"  " + 0.005*"médical"
Topic: 5 
Words: 0.014*"épidémie" + 0.008*"population" + 0.008*"n" + 0.008*"antibiotique" + 0.006*"France" + 0.006*"eau" + 0.006*"prendre" + 0.00

In [38]:
#évaluation modèle
for index, score in sorted(lda_model[corpus[50]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.9844813346862793	 
Topic: 0.032*"d" + 0.019*"l" + 0.016*"épidémie" + 0.009*"n" + 0.007*"devoir"


Topic: 0 Word: 0.002*"faire" + 0.002*"vouloir" + 0.002*"impôt" + 0.002*"suicide" + 0.002*"contribuable" + 0.002*"provoquer" + 0.002*"gagne" + 0.002*"quitter" + 0.002*"crédit" + 0.002*"déjection"
Topic: 1 Word: 0.002*"revoir" + 0.002*"allocation" + 0.002*"falloir" + 0.002*"médecin" + 0.002*"   " + 0.002*"jamais" + 0.002*"virus" + 0.002*"	 " + 0.001*"pognon" + 0.001*"dingue"
Topic: 2 Word: 0.004*"cancer" + 0.003*"maladie" + 0.002*"chronique" + 0.002*"espèce" + 0.002*"toucher" + 0.002*"véritable" + 0.002*"chasse" + 0.002*"jeune" + 0.002*"animal" + 0.002*"apparition"
Topic: 3 Word: 0.003*"devoir" + 0.002*"année" + 0.002*"  " + 0.002*"taxer" + 0.002*">" + 0.002*"=" + 0.002*"faire" + 0.002*"peur" + 0.002*"comportement" + 0.002*"source"
Topic: 4 Word: 0.003*"falloir" + 0.002*"pouvoir" + 0.002*"cas" + 0.002*"risque" + 0.002*"entreprise" + 0.002*"médical" + 0.002*"l" + 0.002*"etat" + 0.002*"terme" + 0.002*"atteindre"
Topic: 5 Word: 0.002*"l" + 0.002*"agriculture" + 0.002*"guerre" + 0.002*"nanti

In [2]:
# initialize api instance
auth = tw.OAuthHandler(consumer_key='suAN7sV4kjo6rNLQsfuNBzQzu',
                        consumer_secret='shj6l92LoN4RkoaFM0Naap0cBW1MbtFAtUKFs45NRNIXELF1jY')

auth.set_access_token('1259561442149351426-gHA05KhC5Oj16kppzcXt5JPhmruRDw',
                        'dKi2yJfcBRgM65ZUHVzJi85ZzlGfd7gQiGUGQHZ3iOjZJ')
api = tw.API(auth, wait_on_rate_limit=True)


In [14]:
swords=["pandemie","pandémie","epidemie","epidémie","sras","coronavirus"]
OU=" OR "
search_words= OU.join(["#"+x for x in swords])
search_words+=" AND (@gouvernementFR OR @Elysee OR @EmmanuelMacron)"
search_words+=" -filter:links"
search_words+=" AND -filter:retweets"
search_words+=" AND -filter:replies"
search_date="2020-07-01"

In [15]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="fr",
              since=search_date, tweet_mode='extended').items(3000)


In [16]:
print(search_words)

#pandemie OR #pandémie OR #epidemie OR #epidémie OR #sras OR #coronavirus AND (@gouvernementFR OR @Elysee OR @EmmanuelMacron) -filter:links AND -filter:retweets AND -filter:replies


In [17]:
def tweet_recup(tweets):
    tweet_list=[]
    for tweet in tweets:
        tweet_text=tweet.full_text
        tweed_id_str=tweet.id_str
        tweet_created_at=tweet.created_at
        tweet_user_location=tweet.user.location
        tweet_list.append({"tweet_text":tweet_text,
                         "tweed_id_str":tweed_id_str,
                         "tweet_created_at":tweet_created_at,
                         "tweet_user_location":tweet_user_location})
    tweets_df=pd.DataFrame(tweet_list,columns=["tweet_text","tweed_id_str","tweet_created_at","tweet_user_location"])
    return tweets_df

In [18]:
df=tweet_recup(tweets)

In [19]:
df.to_excel("tweets.xlsx")

In [ ]:
###On met ce bloc de côté pour l'instant

"""def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="fr", since=search_date, tweet_mode='extended').items(numTweets)
# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
for tweet in tweet_list:
# Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text
# Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time
        
###Cette partie je n'en ai à priori pas besoin###
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_sahkprotests_tweets.csv'
# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))
    """